In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
all_events = ['Santa_Fe_Shooting', 'Student_Marches' , 'Iran_Deal']
label_folder_path = '../stance_dataset/contentious_data/'
data_folder = '../stance_dataset/contentious_data/'
label_file_extension = '/endtags_weighted.csv'

In [3]:
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [4]:
import json
import os
import re
import sys
import pandas as pd
import os.path as path
import numpy as np
import networkx as nx
from node2vec import Node2Vec

In [5]:
df = pd.read_csv("/content/drive/MyDrive/contentious_pairs_with_gpt_ada_embeddings .csv")
df = df.drop(columns=['Event1', 'Event2'])
df['np_target_text_ada_embedding'] = df['target_text_ada_embedding'].apply(eval).apply(np.array)
df['np_response_text_ada_embedding'] = df['response_text_ada_embedding'].apply(eval).apply(np.array)

print(np.shape(df['np_target_text_ada_embedding'][0]))
print(len(df['response_id']))
df

(1536,)
3702


,response_id,event,target_id,interaction_type,label,label_expanded,Confidence_Level,response_text,target_text,truncated,response_text_ada_embedding,target_text_ada_embedding,np_target_text_ada_embedding,np_response_text_ada_embedding
0,997626508050157568,Santa_Fe_Shooting,997598447376175104,Quote,Support,Implicit_Support,1.0,"Seriously, wtf is wrong with our political sys...",More children have been killed in schools this...,False,"[-0.025092221796512604, -0.013276644051074982,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.025092221796512604, -0.013276644051074982,..."
1,997575042027458561,Santa_Fe_Shooting,997573240380968961,Quote,Denial,Explicit_Denial,1.0,Ma calls BS! https://t.co/bodEWN5Q4C,Former GOP Rep. Jason Chaffetz: 'Politically c...,False,"[-0.033440690487623215, 0.007639116141945124, ...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.033440690487623215, 0.007639116141945124, ..."
2,997540582846271494,Santa_Fe_Shooting,997535659870117888,Quote,Denial,Explicit_Denial,1.0,"On average, there’s one fake stat about school...","On average, that’s one school shooting every w...",False,"[-0.023548724129796028, 0.022269316017627716, ...","[-0.0044773295521736145, -0.014855715446174145...","[-0.0044773295521736145, -0.014855715446174145...","[-0.023548724129796028, 0.022269316017627716, ..."
3,1018569947817992192,General_Terms,1017909301040635904,Quote,Denial,Implicit_Denial,1.0,ONE MIGHT BE MADE UP AND NOT REAL. NOT SURE. S...,I’m so confused... - When we were attacked on ...,False,"[0.004989785607904196, -0.004917326848953962, ...","[-0.013004736974835396, -0.01780531369149685, ...","[-0.013004736974835396, -0.01780531369149685, ...","[0.004989785607904196, -0.004917326848953962, ..."
4,1019395289575239680,General_Terms,1017919759474622464,Quote,Denial,Explicit_Denial,1.0,"False, adult friendships are M-F, 9-5 https://...",Adult friendships https://t.co/Cn3r9l4pZJ,False,"[0.023379407823085785, 0.010180916637182236, -...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.023379407823085785, 0.010180916637182236, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,998727686062297088,Santa_Fe_Shooting,998641852202012672,Quote,Support,Implicit_Support,2.0,"Obviously, this is fake news because France ha...",#LeftistTerrorism #LeftistSedition #WhineAndCh...,False,"[-0.012956563383340836, 0.0036507989279925823,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.012956563383340836, 0.0036507989279925823,..."
3698,999033162209443841,Santa_Fe_Shooting,999032790304854021,Reply,Support,Implicit_Support,2.0,@Goss30Goss There is no such thing as a Consti...,Our founding fathers did not create a country ...,False,"[-0.023475689813494682, 0.022063223645091057, ...","[0.0012622424401342869, 0.0014467427972704172,...","[0.0012622424401342869, 0.0014467427972704172,...","[-0.023475689813494682, 0.022063223645091057, ..."
3699,999044683509714944,Santa_Fe_Shooting,999009592867778560,Reply,Denial,Explicit_Denial,2.0,@teamtrace the trace is a @MikeBloomberg shell...,Police confronted the Santa Fe gunman four min...,False,"[-0.0470198318362236, -0.019923418760299683, -...","[-0.01337257120758295, 0.013017261400818825, -...","[-0.01337257120758295, 0.013017261400818825, -...","[-0.0470198318362236, -0.019923418760299683, -..."
3700,999366932024393728,Santa_Fe_Shooting,998952797637890048,Quote,Denial,Implicit_Denial,2.0,This must be a tough position for the parents ...,"“My son, to me, is not a criminal, he’s a vict...",False,"[-0.005462398752570152, -0.00494138989597559, ...","[-0.018083559349179268, 0.016592662781476974, ...","[-0.018083559349179268, 0.016592662781476974, ...","[-0.005462398752570152, -0.00494138989597559, ..."


In [6]:
node_embeddings = np.load('/content/drive/MyDrive/node_embeddings.npy')

In [7]:
import json

with open('/content/drive/MyDrive/id_2_alias.json', 'r') as f:
    id_2_alias = json.load(f)

id_2_alias

{'321631653': 0,
 '18655355': 1,
 '2467720274': 2,
 '3176702526': 3,
 '24939978': 4,
 '1646856415': 5,
 '2932942216': 6,
 '766475610059317248': 7,
 '2909998207': 8,
 '59621769': 9,
 '15537302': 10,
 '908585996': 11,
 '295731821': 12,
 '17564591': 13,
 '966354844754894848': 14,
 '19417492': 15,
 '803694179079458816': 16,
 '58007320': 17,
 '2353605901': 18,
 '117718572': 19,
 '875029068223315968': 20,
 '929766290323058688': 21,
 '35218566': 22,
 '2619593276': 23,
 '36711022': 24,
 '46756474': 25,
 '31059230': 26,
 '27194034': 27,
 '4124770811': 28,
 '2964174789': 29,
 '827641495125688321': 30,
 '69176047': 31,
 '956702734920564736': 32,
 '430106077': 33,
 '1309193942': 34,
 '2745534376': 35,
 '1015843266384859141': 36,
 '3241707920': 37,
 '3836645961': 38,
 '1696503140': 39,
 '477342040': 40,
 '890585748315000832': 41,
 '69004966': 42,
 '934859209279238148': 43,
 '19877186': 44,
 '775899392268177409': 45,
 '1254645558': 46,
 '960994743357292544': 47,
 '15486485': 48,
 '792006970525880320

In [8]:
with open('/content/drive/MyDrive/tweetid_to_users_v2.json', 'r') as f:
    tweetid_to_users = json.load(f)

tweetid_to_users

{'1019932049707155457': '2545173024',
 '1020384634150977536': '38142380',
 '1019140242740318209': '610346981',
 '1021257869759516674': '48442290',
 '999045000406151168': '752555380325113856',
 '1021304858794708992': '389495164',
 '1022108284256436224': '3240040266',
 '997704739117780992': '872165839272894464',
 '1021106637426348034': '553600107',
 '993945299604385794': '202952121',
 '1018484688770490368': '2811494022',
 '1020504407719411714': '769108935726997504',
 '1020269383455735808': '944372156',
 '1021278772794884096': '17006157',
 '977981852420943872': '885601772',
 '1021338975754162176': '516786985',
 '994278930776403968': '2869120412',
 '1018197208967942145': '117718572',
 '998233366150418437': '4733728937',
 '1021385145146736642': '949739194166886400',
 '997705773227556865': '581971753',
 '980335722946334720': '11184122',
 '1000211355528380416': '19080151',
 '1018696685831090177': '996133233065713665',
 '1021777783280693249': '786087060',
 '1019653910183673856': '225265639',
 

In [9]:
df0 = pd.DataFrame(columns=['response_id','response_node', 'event', 'target_node', 'target_id','interaction_type', 'label', 'label_expanded', 'confidence_level', 'response_text', 'target_text', 'truncated', 'np_response_text_ada_embedding', 'np_target_text_ada_embedding', 'present_in_train', 'response_node_embedding', 'target_node_embedding'])

In [10]:
count = 0
for index, row in df.iterrows():
    try:
        response_id = row['response_id']
        target_id = row['target_id']
        response_node = tweetid_to_users[str(response_id)]
        target_node = tweetid_to_users[str(target_id)]
        event = row['event']
        interaction_type = row['interaction_type']
        label = row['label']
        label_expanded = row['label_expanded']
        confidence_level = row['Confidence_Level']
        response_text = row['response_text']
        target_text = row['target_text']
        truncated = row['truncated']
        np_response_text_ada_embedding = row['np_response_text_ada_embedding']
        np_target_text_ada_embedding = row['np_target_text_ada_embedding']
        present_in_train = float('nan')
        response_node_embedding = node_embeddings[id_2_alias[response_node]]
        target_node_embedding = node_embeddings[id_2_alias[target_node]]
        new_row = { 'response_id':response_id,
                    'response_node': response_node,
                    'event': event,
                    'target_node': target_node,
                    'target_id':target_id,
                    'interaction_type': interaction_type,
                    'label': label,
                    'label_expanded': label_expanded,
                    'confidence_level':confidence_level,
                    'response_text':response_text,
                    'target_text':target_text,
                    'truncated': truncated,
                    'np_response_text_ada_embedding':np_response_text_ada_embedding,
                    'np_target_text_ada_embedding':np_target_text_ada_embedding,
                    'present_in_train':present_in_train,
                    'response_node_embedding':response_node_embedding,
                    'target_node_embedding':target_node_embedding}
        #print("done")
        #df0.append(new_row, ignore_index=True)
        df0 = pd.concat([df0, pd.DataFrame([new_row])], ignore_index=True)
    except:
        count +=1
        continue

Streaming output truncated to the last 5000 lines.
<ipython-input-10-d9d9430a7b0b>:40: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df0 = pd.concat([df0, pd.DataFrame([new_row])], ignore_index=True)
<ipython-input-10-d9d9430a7b0b>:40: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df0 = pd.concat([df0, pd.DataFrame([new_row])], ignore_index=True)
<ipython-input-10-d9d9430a7b0b>:40: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df0 = pd.concat([df0, pd.DataFrame([new_row])], ignore_index=True)
<ipython-input-10-d9d9430a7b0b>:40: FutureWarning: In a future version, object-dtype columns with all-bool values w

In [11]:
print("Number of examples skipped:",count)

Number of examples skipped: 401


In [12]:
df2 = df0
df2

,response_id,response_node,event,target_node,target_id,interaction_type,label,label_expanded,confidence_level,response_text,target_text,truncated,np_response_text_ada_embedding,np_target_text_ada_embedding,present_in_train,response_node_embedding,target_node_embedding
0,997626508050157568,18655355,Santa_Fe_Shooting,321631653,997598447376175104,Quote,Support,Implicit_Support,1.0,"Seriously, wtf is wrong with our political sys...",More children have been killed in schools this...,False,"[-0.025092221796512604, -0.013276644051074982,...","[-0.008838048204779625, -0.003347944701090455,...",NaN,"[-0.16415407, -0.14126946, -0.39169663, 0.3252...","[-0.36954, -0.030162731, -0.3114044, 0.0035475..."
1,997575042027458561,3176702526,Santa_Fe_Shooting,2467720274,997573240380968961,Quote,Denial,Explicit_Denial,1.0,Ma calls BS! https://t.co/bodEWN5Q4C,Former GOP Rep. Jason Chaffetz: 'Politically c...,False,"[-0.033440690487623215, 0.007639116141945124, ...","[-0.027197668328881264, 0.0034390492364764214,...",NaN,"[-0.19227098, -0.0046309885, -1.2232333, -0.32...","[-0.031895787, 0.023069244, -1.1588105, -0.234..."
2,997540582846271494,1646856415,Santa_Fe_Shooting,24939978,997535659870117888,Quote,Denial,Explicit_Denial,1.0,"On average, there’s one fake stat about school...","On average, that’s one school shooting every w...",False,"[-0.023548724129796028, 0.022269316017627716, ...","[-0.0044773295521736145, -0.014855715446174145...",NaN,"[0.119322725, 0.14976633, 0.15514494, -0.16940...","[0.12821425, 0.12016403, 0.14294168, -0.164117..."
3,1018569947817992192,766475610059317248,General_Terms,2932942216,1017909301040635904,Quote,Denial,Implicit_Denial,1.0,ONE MIGHT BE MADE UP AND NOT REAL. NOT SURE. S...,I’m so confused... - When we were attacked on ...,False,"[0.004989785607904196, -0.004917326848953962, ...","[-0.013004736974835396, -0.01780531369149685, ...",NaN,"[-0.1845746, 0.20783782, 0.20089266, 0.2106569...","[-0.13405992, 0.23468576, 0.17587437, 0.236973..."
4,1019395289575239680,59621769,General_Terms,2909998207,1017919759474622464,Quote,Denial,Explicit_Denial,1.0,"False, adult friendships are M-F, 9-5 https://...",Adult friendships https://t.co/Cn3r9l4pZJ,False,"[0.023379407823085785, 0.010180916637182236, -...","[0.01642121560871601, 0.0012021968141198158, 0...",NaN,"[-0.13462359, -0.07424727, 0.024345808, 0.1427...","[-0.1618585, -0.10726309, -0.06599967, 0.12543..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3296,998727686062297088,23194229,Santa_Fe_Shooting,899115298997055491,998641852202012672,Quote,Support,Implicit_Support,2.0,"Obviously, this is fake news because France ha...",#LeftistTerrorism #LeftistSedition #WhineAndCh...,False,"[-0.012956563383340836, 0.0036507989279925823,...","[-0.029683180153369904, -0.013163618743419647,...",NaN,"[-0.12518278, 0.16028, -0.13849989, 0.06380284...","[-0.11049661, 0.14310046, -0.05622909, 0.07583..."
3297,999033162209443841,807692474684579840,Santa_Fe_Shooting,2869858477,999032790304854021,Reply,Support,Implicit_Support,2.0,@Goss30Goss There is no such thing as a Consti...,Our founding fathers did not create a country ...,False,"[-0.023475689813494682, 0.022063223645091057, ...","[0.0012622424401342869, 0.0014467427972704172,...",NaN,"[-0.14338258, -0.13098313, 0.13560386, 0.19197...","[-0.20258336, -0.11503259, -0.0454443, 0.06728..."
3298,999044683509714944,994750018300018688,Santa_Fe_Shooting,3243500510,999009592867778560,Reply,Denial,Explicit_Denial,2.0,@teamtrace the trace is a @MikeBloomberg shell...,Police confronted the Santa Fe gunman four min...,False,"[-0.0470198318362236, -0.019923418760299683, -...","[-0.01337257120758295, 0.013017261400818825, -...",NaN,"[-0.16563505, -0.19310482, -0.1267558, 0.14354...","[-0.124195896, -0.14969493, -0.121856034, 0.14..."
3299,999366932024393728,59213861,Santa_Fe_Shooting,424385350,998952797637890048,Quote,Denial,Implicit_Denial,2.0,This must be a tough position for the parents ...,"“My son, to me, is not a

In [13]:
df = df.merge(df2[['response_id', 'response_node' , 'target_node' ]], on="response_id", how = "right")
df

,response_id,event,target_id,interaction_type,label,label_expanded,Confidence_Level,response_text,target_text,truncated,response_text_ada_embedding,target_text_ada_embedding,np_target_text_ada_embedding,np_response_text_ada_embedding,response_node,target_node
0,997626508050157568,Santa_Fe_Shooting,997598447376175104,Quote,Support,Implicit_Support,1.0,"Seriously, wtf is wrong with our political sys...",More children have been killed in schools this...,False,"[-0.025092221796512604, -0.013276644051074982,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.025092221796512604, -0.013276644051074982,...",18655355,321631653
1,997575042027458561,Santa_Fe_Shooting,997573240380968961,Quote,Denial,Explicit_Denial,1.0,Ma calls BS! https://t.co/bodEWN5Q4C,Former GOP Rep. Jason Chaffetz: 'Politically c...,False,"[-0.033440690487623215, 0.007639116141945124, ...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.033440690487623215, 0.007639116141945124, ...",3176702526,2467720274
2,997540582846271494,Santa_Fe_Shooting,997535659870117888,Quote,Denial,Explicit_Denial,1.0,"On average, there’s one fake stat about school...","On average, that’s one school shooting every w...",False,"[-0.023548724129796028, 0.022269316017627716, ...","[-0.0044773295521736145, -0.014855715446174145...","[-0.0044773295521736145, -0.014855715446174145...","[-0.023548724129796028, 0.022269316017627716, ...",1646856415,24939978
3,1018569947817992192,General_Terms,1017909301040635904,Quote,Denial,Implicit_Denial,1.0,ONE MIGHT BE MADE UP AND NOT REAL. NOT SURE. S...,I’m so confused... - When we were attacked on ...,False,"[0.004989785607904196, -0.004917326848953962, ...","[-0.013004736974835396, -0.01780531369149685, ...","[-0.013004736974835396, -0.01780531369149685, ...","[0.004989785607904196, -0.004917326848953962, ...",766475610059317248,2932942216
4,1019395289575239680,General_Terms,1017919759474622464,Quote,Denial,Explicit_Denial,1.0,"False, adult friendships are M-F, 9-5 https://...",Adult friendships https://t.co/Cn3r9l4pZJ,False,"[0.023379407823085785, 0.010180916637182236, -...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.023379407823085785, 0.010180916637182236, -...",59621769,2909998207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3296,998727686062297088,Santa_Fe_Shooting,998641852202012672,Quote,Support,Implicit_Support,2.0,"Obviously, this is fake news because France ha...",#LeftistTerrorism #LeftistSedition #WhineAndCh...,False,"[-0.012956563383340836, 0.0036507989279925823,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.012956563383340836, 0.0036507989279925823,...",23194229,899115298997055491
3297,999033162209443841,Santa_Fe_Shooting,999032790304854021,Reply,Support,Implicit_Support,2.0,@Goss30Goss There is no such thing as a Consti...,Our founding fathers did not create a country ...,False,"[-0.023475689813494682, 0.022063223645091057, ...","[0.0012622424401342869, 0.0014467427972704172,...","[0.0012622424401342869, 0.0014467427972704172,...","[-0.023475689813494682, 0.022063223645091057, ...",807692474684579840,2869858477
3298,999044683509714944,Santa_Fe_Shooting,999009592867778560,Reply,Denial,Explicit_Denial,2.0,@teamtrace the trace is a @MikeBloomberg shell...,Police confronted the Santa Fe gunman four min...,False,"[-0.0470198318362236, -0.019923418760299683, -...","[-0.01337257120758295, 0.013017261400818825, -...","[-0.01337257120758295, 0.013017261400818825, -...","[-0.0470198318362236, -0.019923418760299683, -...",994750018300018688,3243500510
3299,999366932024393728,Santa_Fe_Shooting,998952797637890048,Quote,Denial,Implicit_Denial,2.0,This must be a tough position for the parents ...,"“My son, to me, is not a criminal, he’s a vict...",False,"[-0.005462398752570152, -0.004

In [14]:
df.to_csv('/content/drive/MyDrive/final_data_ISB.csv', index=False)

In [15]:
nodes_of_interest = set(df['target_node'].astype(str)) | ( set(df['response_node'].astype(str)))

len(nodes_of_interest)

4944

In [16]:
def add_user_feature_count_to_dict(user_dict, user, feature, count):
        try:
            if user not in user_dict:
                user_dict[user] = {}

            user_feature_count =  user_dict[user]

            if feature not in user_feature_count:
                user_feature_count[feature] = int(count)
            else:
                user_feature_count[feature] += int(count)
        except:
            print('error', feature, count)

def add_feature_count_to_dict(feature_dict, feature, count):
    try:
        if feature not in feature_dict:
            feature_dict[feature] = int(count)
        else:
            feature_dict[feature] += int(count)

    except:
        print('error', feature, count)

event_data = {}
user_tweet_count = {}
user_endtag_count = {}
endtag_count = {}

data_folder = '/content/drive/MyDrive/contentious_data'
for event in all_events: #all_events: #['Santa_Fe_Shooting', 'Student_Marches' , 'Iran_Deal']:

    folder_path =  data_folder + '/' + event + '/'
    with open(path.join(folder_path, 'retweets_weighted.csv') , 'r') as f_read: #retweets_weighted, endtags
        for line in f_read:
            parts = line.strip().split(',')

            if len(parts) ==3:
                user, endtag, count = parts
                user = str(user)
                if user in nodes_of_interest:

                    add_feature_count_to_dict(endtag_count, endtag, count)
                    add_feature_count_to_dict(user_tweet_count, user, count)
                    add_user_feature_count_to_dict(user_endtag_count, user, endtag, count)

    print(event, len(user_tweet_count), len(user_endtag_count), len(endtag_count))

Santa_Fe_Shooting 582 582 886418
Student_Marches 776 776 1081014
Iran_Deal 1230 1230 1358677


In [17]:
endtag_count

{'491827156': 1,
 '452257946': 21,
 '780195510779056129': 2,
 '3158213423': 1,
 '746116280856514560': 66,
 '389786453': 3,
 '2553408986': 12,
 '26008843': 1,
 '3052092737': 4,
 '2897860973': 1,
 '3301266202': 28,
 '1360140709': 78,
 '2429410915': 68,
 '2825906244': 3,
 '951819078456545280': 1,
 '332032129': 3,
 '1542432121': 162,
 '145840139': 1,
 '615724020': 2,
 '975771699940651008': 1,
 '2424504228': 14,
 '389781019': 1,
 '16798086': 31,
 '3223107533': 3,
 '926760505': 1,
 '162547325': 6,
 '3176793078': 1,
 '2754067426': 2,
 '356944528': 1,
 '895160071243616256': 1,
 '2464007634': 24,
 '2183113232': 48,
 '510643018': 6,
 '228134082': 32,
 '321881336': 3,
 '816326258543865856': 21,
 '940127748': 1,
 '759497349634879488': 45,
 '261071188': 1,
 '960327149159927808': 11,
 '860691374': 2,
 '553077633': 1,
 '2967943317': 68,
 '2313024324': 11,
 '183433807': 4,
 '374673775': 161,
 '934108670325067776': 1,
 '930564325756063745': 288,
 '829724275900833793': 9,
 '797558899431718912': 137,
 '2

In [18]:
def create_vector(user_property_count, tweet_user_index, N_TOP, binarize_count):
    user_hashtag_weighted_matrix = np.zeros((len(tweet_user_index), N_TOP))

    hashtags_users = {}
    hashtag_index = {}
    index_to_tag = {}
    for user, hashtags_count in user_property_count.items():
        for hashtag, count in hashtags_count.items():
            if hashtag not in hashtags_users:
                hashtags_users[hashtag] = []

            hashtags_users[hashtag].append(user)


    print('hashtags_users: ', len(hashtags_users))

    sorted_hashtags = sorted(hashtags_users.items(), key=lambda x: len(x[1]), reverse=True)

    for tag, users in sorted_hashtags[0:N_TOP]:
        for user in users:
            user_index = tweet_user_index[user]
            if tag not in hashtag_index:
                hashtag_index[tag] = len(hashtag_index)
                index_to_tag[len(hashtag_index) - 1] = tag

            tag_index = hashtag_index[tag]

            if tag in user_property_count[user]:
                user_hastag_time_used = user_property_count[user][tag]

                if binarize_count:
                    user_hashtag_weighted_matrix[user_index, tag_index] = 1
                else:
                    user_hashtag_weighted_matrix[user_index, tag_index] = user_hastag_time_used

#     if normalize_row:
#         normalized_user_hashtag_weighted_matrix = TextProcessingHelpers.row_normalize(user_hashtag_weighted_matrix)
#     else:
    normalized_user_hashtag_weighted_matrix = user_hashtag_weighted_matrix

    return normalized_user_hashtag_weighted_matrix, hashtag_index, index_to_tag


tweet_user_index = {}
tweet_user_index_to_id = {}
unique_users_index = {}
users_index_to_id = {}

for user, count in user_tweet_count.items():
    if user not in tweet_user_index:
        tweet_user_index_length = len(tweet_user_index)
        tweet_user_index[user] = tweet_user_index_length
        tweet_user_index_to_id[tweet_user_index_length] = user

    if user not in unique_users_index:
        unique_users_index_length = len(unique_users_index)
        unique_users_index[user] = unique_users_index_length
        users_index_to_id[unique_users_index_length] = user


# print('Creating retweet weight matrix')
# user_retweet_weighted_matrix, __, __ = create_vector(user_retweet_count,
#                                                      tweet_user_index,
#                                                      N_TOP_RETWEET,
#                                                      binarize_count)

print('Creating hashtag weight matrix')
user_hashtag_weighted_matrix, hashtag_index, index_to_tag = create_vector(user_endtag_count,
                                                                          tweet_user_index,
                                                                          N_TOP = 500,
                                                                          binarize_count = False)

Creating hashtag weight matrix
hashtags_users:  1358677


In [19]:
tweet_user_index

{'53692193': 0,
 '1317129865': 1,
 '855007514621911040': 2,
 '878087280321335298': 3,
 '25073877': 4,
 '727264708776980480': 5,
 '3493563087': 6,
 '292929271': 7,
 '810619093749559296': 8,
 '17564591': 9,
 '26642006': 10,
 '38029205': 11,
 '807095': 12,
 '6473022': 13,
 '187428929': 14,
 '17261066': 15,
 '14247236': 16,
 '964032914626359296': 17,
 '149249831': 18,
 '27032207': 19,
 '21059255': 20,
 '246939630': 21,
 '19568591': 22,
 '469194846': 23,
 '21829541': 24,
 '983766825610694656': 25,
 '16041234': 26,
 '701725963': 27,
 '1917731': 28,
 '7702542': 29,
 '28785486': 30,
 '817382752731463680': 31,
 '1915033663': 32,
 '133938408': 33,
 '78515008': 34,
 '1589251639': 35,
 '32871086': 36,
 '958064770019741696': 37,
 '797229459803926528': 38,
 '18510860': 39,
 '132339474': 40,
 '24165761': 41,
 '1462969034': 42,
 '812135885328236551': 43,
 '817661098988019712': 44,
 '166751745': 45,
 '3260518932': 46,
 '15907183': 47,
 '755057897583804416': 48,
 '20772763': 49,
 '334079075': 50,
 '2411

In [21]:
reverse_tweet_user_index = {value: key for key, value in tweet_user_index.items()}

In [22]:
reverse_tweet_user_index

{0: '53692193',
 1: '1317129865',
 2: '855007514621911040',
 3: '878087280321335298',
 4: '25073877',
 5: '727264708776980480',
 6: '3493563087',
 7: '292929271',
 8: '810619093749559296',
 9: '17564591',
 10: '26642006',
 11: '38029205',
 12: '807095',
 13: '6473022',
 14: '187428929',
 15: '17261066',
 16: '14247236',
 17: '964032914626359296',
 18: '149249831',
 19: '27032207',
 20: '21059255',
 21: '246939630',
 22: '19568591',
 23: '469194846',
 24: '21829541',
 25: '983766825610694656',
 26: '16041234',
 27: '701725963',
 28: '1917731',
 29: '7702542',
 30: '28785486',
 31: '817382752731463680',
 32: '1915033663',
 33: '133938408',
 34: '78515008',
 35: '1589251639',
 36: '32871086',
 37: '958064770019741696',
 38: '797229459803926528',
 39: '18510860',
 40: '132339474',
 41: '24165761',
 42: '1462969034',
 43: '812135885328236551',
 44: '817661098988019712',
 45: '166751745',
 46: '3260518932',
 47: '15907183',
 48: '755057897583804416',
 49: '20772763',
 50: '334079075',
 51: '

In [23]:
print(np.shape(user_hashtag_weighted_matrix))
graph=nx.from_numpy_array(np.dot(user_hashtag_weighted_matrix, user_hashtag_weighted_matrix.T))

(1230, 500)


In [24]:
nx_graph = nx.Graph(graph)
print(nx_graph)

Graph with 1230 nodes and 153693 edges


In [28]:
import numpy as np

#load your embeddings here
node_embeddings_deepwalk = np.load('/content/drive/MyDrive/node_embeddings_node_2_vec_similarity.npy')

# Print the array
print(node_embeddings_deepwalk)
print(np.shape(node_embeddings_deepwalk))
node_embeddings_deepwalk[0]

[[-1.29172895e-02 -1.89467931e+00  5.38042367e-01 ... -5.98128498e-01
   2.39378405e+00  5.65739095e-01]
 [ 6.11948848e-01 -4.09329981e-01  2.24881425e-01 ... -6.44837022e-01
   2.22075772e+00  9.11846578e-01]
 [-1.10918462e+00 -1.03814125e+00  7.45282650e-01 ... -1.14743042e+00
   6.28776550e-01  4.11977619e-01]
 ...
 [-1.46596844e-03 -1.26457942e+00  2.34216005e-01 ...  3.36004913e-01
   2.04829144e+00  1.55490816e+00]
 [-2.15006447e+00 -2.05588531e+00  1.59331501e+00 ...  1.09673893e+00
   4.04085338e-01 -5.50472558e-01]
 [-7.63630390e-01 -5.32063723e-01 -1.12677562e+00 ... -1.31933165e+00
   3.72156382e-01 -9.12586033e-01]]
(5186, 128)


array([-0.01291729, -1.8946793 ,  0.53804237,  1.3048033 , -0.45961025,
        0.9205481 ,  0.11403947,  2.074149  , -1.4015977 , -0.1602076 ,
       -0.16384195, -1.1430645 ,  0.9060119 ,  0.27203503, -0.03658624,
       -0.22299047, -1.3110509 ,  0.20071709, -0.7674612 , -0.12663668,
        1.401846  ,  0.60417616,  0.59172916, -1.0038841 ,  0.5349817 ,
       -1.3956587 , -0.13245355,  0.12935463, -0.7912642 ,  1.0176091 ,
        0.5029314 , -1.0482635 , -0.73717093,  1.273871  , -0.7577297 ,
        0.2406318 ,  0.7372727 , -1.5734036 , -0.2865742 ,  0.70868516,
       -0.8054817 ,  1.9818277 , -0.4337848 ,  0.69398713, -0.7295908 ,
        0.58996135, -1.0040507 , -0.42436844, -0.58222115,  0.32610428,
       -1.3968666 , -0.17821464, -0.06962667, -0.51919186, -0.52970266,
        0.1270808 ,  1.2931515 , -0.25520912, -0.39527592,  0.32419276,
        0.57487845,  0.8868701 , -0.33864725, -0.47395423, -0.2893033 ,
       -0.11087172, -0.8307094 ,  0.07733116, -0.7569955 , -0.34

In [31]:
import json

with open('/content/drive/MyDrive/alias_2_id.json', 'r') as file:
    reverse_tweet_user_index = json.load(file)

print(reverse_tweet_user_index)


{'0': '321631653', '1': '18655355', '2': '2467720274', '3': '3176702526', '4': '24939978', '5': '1646856415', '6': '2932942216', '7': '766475610059317248', '8': '2909998207', '9': '59621769', '10': '15537302', '11': '908585996', '12': '295731821', '13': '17564591', '14': '966354844754894848', '15': '19417492', '16': '803694179079458816', '17': '58007320', '18': '2353605901', '19': '117718572', '20': '875029068223315968', '21': '929766290323058688', '22': '35218566', '23': '2619593276', '24': '36711022', '25': '46756474', '26': '31059230', '27': '27194034', '28': '4124770811', '29': '2964174789', '30': '827641495125688321', '31': '69176047', '32': '956702734920564736', '33': '430106077', '34': '1309193942', '35': '2745534376', '36': '1015843266384859141', '37': '3241707920', '38': '3836645961', '39': '1696503140', '40': '477342040', '41': '890585748315000832', '42': '69004966', '43': '934859209279238148', '44': '19877186', '45': '775899392268177409', '46': '1254645558', '47': '960994743

In [34]:
user_vector = {}
for index, user in reverse_tweet_user_index.items():
    user_vector[user] = node_embeddings_deepwalk[int(index)]

In [35]:
user_vector

{'321631653': array([-0.01291729, -1.8946793 ,  0.53804237,  1.3048033 , -0.45961025,
         0.9205481 ,  0.11403947,  2.074149  , -1.4015977 , -0.1602076 ,
        -0.16384195, -1.1430645 ,  0.9060119 ,  0.27203503, -0.03658624,
        -0.22299047, -1.3110509 ,  0.20071709, -0.7674612 , -0.12663668,
         1.401846  ,  0.60417616,  0.59172916, -1.0038841 ,  0.5349817 ,
        -1.3956587 , -0.13245355,  0.12935463, -0.7912642 ,  1.0176091 ,
         0.5029314 , -1.0482635 , -0.73717093,  1.273871  , -0.7577297 ,
         0.2406318 ,  0.7372727 , -1.5734036 , -0.2865742 ,  0.70868516,
        -0.8054817 ,  1.9818277 , -0.4337848 ,  0.69398713, -0.7295908 ,
         0.58996135, -1.0040507 , -0.42436844, -0.58222115,  0.32610428,
        -1.3968666 , -0.17821464, -0.06962667, -0.51919186, -0.52970266,
         0.1270808 ,  1.2931515 , -0.25520912, -0.39527592,  0.32419276,
         0.57487845,  0.8868701 , -0.33864725, -0.47395423, -0.2893033 ,
        -0.11087172, -0.8307094 ,  0.0

In [36]:
response_node_embeddings = []
target_node_embeddings = []

na_count = 0
for response_node, target_node in zip(df['response_node'], df['target_node']):
    response_node = str(response_node)
    target_node = str(target_node)
#     print(response_node, target_node)
    if response_node in user_vector:
        response_node_embeddings.append(user_vector[response_node])
    else:
        response_node_embeddings.append(np.mean(node_embeddings_deepwalk, axis=0))
        na_count += 1

    if target_node in user_vector:
        target_node_embeddings.append(user_vector[target_node])
    else:
        target_node_embeddings.append(np.mean(node_embeddings_deepwalk, axis=0))
        na_count += 1

print('na_count:', na_count)
na_count: 3828

na_count: 0


In [37]:
df['updated_response_node_embedding'] = response_node_embeddings
df['updated_target_node_embedding'] = target_node_embeddings
df

,response_id,event,target_id,interaction_type,label,label_expanded,Confidence_Level,response_text,target_text,truncated,response_text_ada_embedding,target_text_ada_embedding,np_target_text_ada_embedding,np_response_text_ada_embedding,response_node,target_node,updated_response_node_embedding,updated_target_node_embedding
0,997626508050157568,Santa_Fe_Shooting,997598447376175104,Quote,Support,Implicit_Support,1.0,"Seriously, wtf is wrong with our political sys...",More children have been killed in schools this...,False,"[-0.025092221796512604, -0.013276644051074982,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.008838048204779625, -0.003347944701090455,...","[-0.025092221796512604, -0.013276644051074982,...",18655355,321631653,"[0.61194885, -0.40932998, 0.22488143, 0.388472...","[-0.0129172895, -1.8946793, 0.53804237, 1.3048..."
1,997575042027458561,Santa_Fe_Shooting,997573240380968961,Quote,Denial,Explicit_Denial,1.0,Ma calls BS! https://t.co/bodEWN5Q4C,Former GOP Rep. Jason Chaffetz: 'Politically c...,False,"[-0.033440690487623215, 0.007639116141945124, ...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.027197668328881264, 0.0034390492364764214,...","[-0.033440690487623215, 0.007639116141945124, ...",3176702526,2467720274,"[-0.17848909, -1.8448285, 1.8054861, 0.7048951...","[-1.1091846, -1.0381413, 0.74528265, 1.9173095..."
2,997540582846271494,Santa_Fe_Shooting,997535659870117888,Quote,Denial,Explicit_Denial,1.0,"On average, there’s one fake stat about school...","On average, that’s one school shooting every w...",False,"[-0.023548724129796028, 0.022269316017627716, ...","[-0.0044773295521736145, -0.014855715446174145...","[-0.0044773295521736145, -0.014855715446174145...","[-0.023548724129796028, 0.022269316017627716, ...",1646856415,24939978,"[0.7771124, -0.3952584, -0.515285, -0.27929837...","[0.6408851, -1.5015211, -0.91648525, 0.2109703..."
3,1018569947817992192,General_Terms,1017909301040635904,Quote,Denial,Implicit_Denial,1.0,ONE MIGHT BE MADE UP AND NOT REAL. NOT SURE. S...,I’m so confused... - When we were attacked on ...,False,"[0.004989785607904196, -0.004917326848953962, ...","[-0.013004736974835396, -0.01780531369149685, ...","[-0.013004736974835396, -0.01780531369149685, ...","[0.004989785607904196, -0.004917326848953962, ...",766475610059317248,2932942216,"[-0.6820378, -0.5925498, 0.4861588, 1.8928771,...","[-0.44048476, 0.27823466, 0.21975206, 1.035810..."
4,1019395289575239680,General_Terms,1017919759474622464,Quote,Denial,Explicit_Denial,1.0,"False, adult friendships are M-F, 9-5 https://...",Adult friendships https://t.co/Cn3r9l4pZJ,False,"[0.023379407823085785, 0.010180916637182236, -...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.01642121560871601, 0.0012021968141198158, 0...","[0.023379407823085785, 0.010180916637182236, -...",59621769,2909998207,"[-1.1356885, -2.2024667, 2.2011151, 1.6681736,...","[-0.8448516, -2.2348397, 2.0122194, 0.8253141,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3296,998727686062297088,Santa_Fe_Shooting,998641852202012672,Quote,Support,Implicit_Support,2.0,"Obviously, this is fake news because France ha...",#LeftistTerrorism #LeftistSedition #WhineAndCh...,False,"[-0.012956563383340836, 0.0036507989279925823,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.029683180153369904, -0.013163618743419647,...","[-0.012956563383340836, 0.0036507989279925823,...",23194229,899115298997055491,"[1.2769866, -3.1484814, -0.82575464, 0.1997960...","[0.6509096, -2.9987767, 0.4748726, 1.3316041, ..."
3297,999033162209443841,Santa_Fe_Shooting,999032790304854021,Reply,Support,Implicit_Support,2.0,@Goss30Goss There is no such thing as a Consti...,Our founding fathers did not create a country ...,False,"[-0.023475689813494682, 0.022063223645091057, ...","[0.0012622424401342869, 0.0014467427972704172,...","[0.0012622424401342869, 0.0014467427972704172,...","[-0.023475689813494682, 0.022063223645091057, ...",8076924

In [38]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.pipeline import Pipeline

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
# from xgboost.sklearn import XGBClassifier
# import xgboost as xgb


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier

from numpy import random as rd


def get_text(tweet):
    tweetText = re.sub(r"(?:\@|https?\://)\S+", "", tweet) # r'(?:@[\w_]+)'
    tweetText = tweetText.strip().replace('rt', '').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')

    return tweetText



def get_statce_stats(emb, label):
    y_counts = {}
    y_Xs = {}
    y_indices = {}
    for i, (emb, label) in enumerate(zip(emb, label)):
        if label in y_counts:
            y_counts[label] = y_counts[label]  + 1
        else:
            y_counts[label] = 1

        if label in y_Xs:
            y_Xs[label].append(emb)
        else:
            y_Xs[label] = [emb]

        if label in y_indices:
            y_indices[label].append(i)
        else:
            y_indices[label] = [i]

    print('lables : ', y_counts)

    return y_counts, y_Xs, y_indices

def balance_stance_classes(emb_vec, labels):
    y_counts, y_Xs, y_indices = get_statce_stats(emb_vec, labels)


    majority_label = 0
    minority_label = 1
    majority_count = 0
    for key, val in y_counts.items():
        if val > majority_count:

            minority_label = majority_label
            majority_label = key
            majority_count= val
        else:
            minority_label = key

    new_emb_vec = []
    new_labels = []
    for label in y_counts.keys():
        if label != majority_label:
            minority_label = label

            if y_counts[majority_label] > y_counts[minority_label]:
                additional_samples_count = y_counts[majority_label] - y_counts[minority_label]

                indices = rd.choice(list(y_indices[minority_label]), size=additional_samples_count, replace=True)
                for index in indices:
                    new_emb_vec.append(emb_vec[index])
                    new_labels.append(labels[index])


    y_counts, y_Xs, y_indices  = get_statce_stats(new_emb_vec + emb_vec, new_labels + labels)


    return  new_emb_vec + emb_vec, new_labels + labels



classifier = ('svm12', Pipeline([('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                                       alpha=1e-3, max_iter=15, random_state=42)),
                                            ])
                                        , {'vect__ngram_range': [(1, 1), (1, 2)],
                                          'tfidf__use_idf': (True, False),
                                          'clf-svm__alpha': (1e-2, 1e-3),
                                         })

clf_name, text_clf, __  = classifier

# train_replies, train_sources, train_labels, test_replies, test_sources, test_labels, test_source_reply_user_ids = prepare_data(all_event_data, additional_training_data)

# text_clf.fit(train_replies,  train_labels)


# >>> X = [[0., 0.], [1., 1.]]
# >>> y = [0, 1]
mlp_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

sgd_clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=15, random_state=42)




import numpy as np


In [39]:
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


train_embedding_label = []
test_embedding_label = []
train_count = 0
test_count = 0

X_train_all = []
X_test_all = []

y_train_all = []
y_test_all = []


for event in {'General_Terms', 'Iran_Deal', 'Santa_Fe_Shooting', 'Student_Marches'}:
    df_event_train = df[(df['event'] != event)] #  &
    df_event_test = df[df['event'] == event] #  &

    print(event, 'Train: ', len(df_event_train), 'Test: ',  len(df_event_test))

    X_train = []
    X_test = []
    y_train = []
    y_test = []


#     print(len(df_event_train))


    for response_embedding, target_embedding, response_node_embedding, target_node_embedding,  label in zip(df_event_train['np_response_text_ada_embedding'], df_event_train['np_target_text_ada_embedding'], df_event_train['updated_response_node_embedding'], df_event_train['updated_target_node_embedding'], df_event_train['label']):

#         print(np.shape(response_embedding), np.shape(response_node_embedding), np.shape(target_node_embedding))

        #embedding = np.concatenate([response_embedding , target_embedding, response_node_embedding, target_node_embedding ], axis = 0)
        embedding = np.concatenate([response_node_embedding, target_node_embedding ], axis = 0)
        train_embedding_label.append([embedding, label])

        X_train.append(embedding)
        y_train.append(label)

        train_count += 1


    for response_embedding, target_embedding, response_node_embedding,target_node_embedding, label in zip(df_event_test['np_response_text_ada_embedding'], df_event_test['np_target_text_ada_embedding'],df_event_train['updated_response_node_embedding'], df_event_train['updated_target_node_embedding'],  df_event_test['label']):



        #embedding = np.concatenate([response_embedding , target_embedding, response_node_embedding,target_node_embedding], axis = 0)

        test_embedding_label.append([embedding, label])
        embedding = np.concatenate([response_node_embedding, target_node_embedding ], axis = 0)
        X_test.append(embedding)
        y_test.append(label)



        test_count += 1

#     print(len(X_train), len(y_train), len(X_test), len(y_test))

    classification_data = (X_train)
    data_labels = (y_train)
    balanced_classification_data, balanced_labels = balance_stance_classes(classification_data, data_labels)
    print(' Balanced train: ', len(balanced_classification_data), len(classification_data))

#     clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 3, 2), random_state=1)
    clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=15, random_state=42)
#     clf = RandomForestClassifier(n_estimators=750)

    clf.fit(balanced_classification_data, balanced_labels)
    preds = clf.predict(X_test)

    print(classification_report(y_test, preds))

Santa_Fe_Shooting Train:  2392 Test:  909
lables :  {'Denial': 1325, 'Support': 1067}
lables :  {'Support': 1325, 'Denial': 1325}
 Balanced train:  2650 2392
              precision    recall  f1-score   support

      Denial       0.59      0.39      0.47       567
     Support       0.35      0.54      0.43       342

    accuracy                           0.45       909
   macro avg       0.47      0.47      0.45       909
weighted avg       0.50      0.45      0.45       909

General_Terms Train:  2292 Test:  1009
lables :  {'Support': 992, 'Denial': 1300}
lables :  {'Support': 1300, 'Denial': 1300}
 Balanced train:  2600 2292


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


              precision    recall  f1-score   support

      Denial       0.59      0.71      0.64       592
     Support       0.42      0.29      0.34       417

    accuracy                           0.54      1009
   macro avg       0.50      0.50      0.49      1009
weighted avg       0.52      0.54      0.52      1009

Student_Marches Train:  2768 Test:  533
lables :  {'Support': 1167, 'Denial': 1601}
lables :  {'Support': 1601, 'Denial': 1601}
 Balanced train:  3202 2768
              precision    recall  f1-score   support

      Denial       0.55      0.56      0.56       291
     Support       0.46      0.45      0.46       242

    accuracy                           0.51       533
   macro avg       0.51      0.51      0.51       533
weighted avg       0.51      0.51      0.51       533

Iran_Deal Train:  2451 Test:  850
lables :  {'Support': 1001, 'Denial': 1450}
lables :  {'Support': 1450, 'Denial': 1450}
 Balanced train:  2900 2451
              precision    recall  f1-sc

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [40]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


import numpy as np

train_embedding_label = []
test_embedding_label = []
train_count = 0
test_count = 0

X_train_all = []
X_test_all = []

y_train_all = []
y_test_all = []


for event in {'General_Terms', 'Iran_Deal', 'Santa_Fe_Shooting', 'Student_Marches'}:
    df_event_train = df[(df['event'] != event)] #  &
    df_event_test = df[df['event'] == event] #  &

    print(event, 'Train: ', len(df_event_train), 'Test: ',  len(df_event_test))

    X_train = []
    X_test = []
    y_train = []
    y_test = []


    print(len(df_event_train))


    for response_embedding, target_embedding, label in zip(df_event_train['np_response_text_ada_embedding'], df_event_train['np_target_text_ada_embedding'], df_event_train['label']):

        embedding = np.concatenate([response_embedding , target_embedding, response_node_embedding,target_node_embedding], axis = 0)
        train_embedding_label.append([embedding, label])

        X_train.append(embedding)
        y_train.append(label)

        train_count += 1


    for response_embedding, target_embedding, label in zip(df_event_test['np_response_text_ada_embedding'], df_event_test['np_target_text_ada_embedding'], df_event_test['label']):


        embedding = np.concatenate([response_embedding , target_embedding, response_node_embedding,target_node_embedding], axis = 0)
        test_embedding_label.append([embedding, label])

        X_test.append(embedding)
        y_test.append(label)



        test_count += 1

    print(np.shape(response_embedding), np.shape(target_embedding))


    print(len(X_train), len(y_train), len(X_test), len(y_test))

    classification_data = (X_train)
    data_labels = (y_train)
    balanced_classification_data, balanced_labels = balance_stance_classes(classification_data, data_labels)
    print(' Balanced train: ', len(balanced_classification_data), len(classification_data))

    #clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
    clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=500, random_state=42)
    #clf = RandomForestClassifier(n_estimators=750)

    clf.fit(balanced_classification_data, balanced_labels)
    preds = clf.predict(X_test)

    print(classification_report(y_test, preds))

Santa_Fe_Shooting Train:  2392 Test:  909
2392
(1536,) (1536,)
2392 2392 909 909
lables :  {'Denial': 1325, 'Support': 1067}
lables :  {'Support': 1325, 'Denial': 1325}
 Balanced train:  2650 2392
              precision    recall  f1-score   support

      Denial       0.95      0.30      0.46       567
     Support       0.46      0.97      0.62       342

    accuracy                           0.55       909
   macro avg       0.70      0.64      0.54       909
weighted avg       0.76      0.55      0.52       909

General_Terms Train:  2292 Test:  1009
2292
(1536,) (1536,)
2292 2292 1009 1009
lables :  {'Support': 992, 'Denial': 1300}
lables :  {'Support': 1300, 'Denial': 1300}
 Balanced train:  2600 2292
              precision    recall  f1-score   support

      Denial       0.85      0.48      0.61       592
     Support       0.54      0.88      0.67       417

    accuracy                           0.64      1009
   macro avg       0.69      0.68      0.64      1009
weighted 